In [154]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [155]:
data = pd.read_csv('data_banknote_authentication.csv')
data = np.array(data)
# print(data[0:5])

In [156]:
X, y = data[:,0:-1], data[:,-1].reshape(-1,1)
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.25, random_state=23)
X_train, X_dev, y_train, y_dev  = X_train.T, X_dev.T, y_train.T, y_dev.T
print(f'Input Shape: {X_train.shape}\nOutput Shape: {y_train.shape}')

Input Shape: (4, 1028)
Output Shape: (1, 1028)


In [157]:
# Number of features
n = X_train.shape[0]
# Number of examples
m = X_train.shape[1]
print(f"Number of features: {n}\nNumber of examples: {m}")

Number of features: 4
Number of examples: 1028


In [158]:
# Sigmoid helper function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [159]:
# Initialize training parameters W and b
def initialize_params():
    W = np.random.randn(1,n)
    b = 0
    return W, b

In [160]:
# Forward pass
def forward_prop(X, W, b):
    z = W @ X + b
    y_hat = sigmoid(z)
    return y_hat

In [161]:
# Compute the cost (negative log likelihood)
def compute_cost(y_hat, y):
    epsilon = 1e-8
    J = np.sum(- (y * np.log(y_hat + epsilon)) - ((1 - y) * np.log(1 - y_hat + epsilon)))
    return J

In [162]:
# Backprop pass
def backprop(y, y_hat, X):
    dJ__dy_hat = (-y / y_hat) + (1 - y)/(1 - y_hat)
    dy_hat__dz = y_hat * (1 - y_hat)
    dz__dw = X
    dz__db = 1

    dJ__dw = (dJ__dy_hat * dy_hat__dz) @ dz__dw.T / m
    dJ__db = np.sum(dJ__dy_hat * dy_hat__dz) * dz__db / m

    return dJ__dw, dJ__db

In [163]:
# Update parameters
def update_params(W, b, dJdW, dJdb, alpha):
    W = W - alpha * dJdW
    b = b - alpha * dJdb
    return W, b

In [164]:
# Train the logistic regression classifier from pieces above
def logistic_regression(X, y, num_iterations=1000, alpha=0.01):

    W, b = initialize_params()

    for i in range(num_iterations + 1):

        y_hat = forward_prop(X, W, b)
        J = compute_cost(y_hat, y)
        if i % (num_iterations // 10) == 0:
            print(f'Iter {i} cost: {J}')
        dJdW, dJdb = backprop(y, y_hat, X)
        W, b = update_params(W, b, dJdW, dJdb, alpha)

    return W, b

In [165]:
W, b = logistic_regression(X_train, y_train, num_iterations=10000, alpha=0.0003)

Iter 0 cost: 6160.17124189865
Iter 1000 cost: 1942.517830025484
Iter 2000 cost: 796.1339917638447
Iter 3000 cost: 535.9986130938719
Iter 4000 cost: 411.5338656378153
Iter 5000 cost: 338.0338311860845
Iter 6000 cost: 291.74433624187975
Iter 7000 cost: 260.6670614368115
Iter 8000 cost: 238.42891495733028
Iter 9000 cost: 221.6213721974674
Iter 10000 cost: 208.35915683187005


In [166]:
def compute_accuracy(y, y_preds):
    total = y_preds.shape[1]
    correct = np.sum(abs(y_preds - y) < 0.5)
    return correct / total

In [167]:
y_dev_preds = forward_prop(X_dev, W, b)

accuracy = compute_accuracy(y_dev, y_dev_preds)
print(f'Dev set accuracy: {accuracy}')

Dev set accuracy: 0.9212827988338192
